In [209]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)
import numpy as np
import statistics

sp500_history = pd.read_csv('./sp500_history.csv')
sp500_history['date'] = pd.to_datetime(sp500_history['date']) 
sp500_history = sp500_history[['date', 'value', 'variable']]

start_date = datetime.strptime('2016-1-1', '%Y-%m-%d')
end_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history['date'] > start_date) & (sp500_history['date'] <= end_date)
sp500_history = sp500_history.loc[mask]

# Sort
sp500_history.sort_values(['date', 'variable'], ascending=[True, True])

,date,value,variable
261,2016-01-05,WLTW,added_ticker
260,2016-01-05,FOSL,removed_ticker
262,2016-01-19,EXR,added_ticker
263,2016-01-19,CB,removed_ticker
264,2016-02-01,FRT,added_ticker
...,...,...,...
453,2019-12-23,ZBRA,added_ticker
454,2019-12-23,STE,added_ticker
449,2019-12-23,AMG,removed_ticker
451,2019-12-23,MAC,removed_ticker


In [207]:
calendar = pd.read_excel('./Nasdaq_Trading_Calendar.xlsx', sheet_name=None)
rebal_dates = []
for year in calendar:
    sheet = calendar[year]
    mask = sheet['S&P Indexes Rebalance S&P 500, S&P 400, and S&P 600'] == 1
    year_rebal_dates = sheet.loc[mask]
    for date in year_rebal_dates['Date'].values:
        rebal_dates.append(date)


In [213]:
rebal_dates

[numpy.datetime64('2020-03-20T00:00:00.000000000'),
 numpy.datetime64('2020-06-19T00:00:00.000000000'),
 numpy.datetime64('2020-09-18T00:00:00.000000000'),
 numpy.datetime64('2020-12-18T00:00:00.000000000'),
 numpy.datetime64('2019-03-15T00:00:00.000000000'),
 numpy.datetime64('2019-06-21T00:00:00.000000000'),
 numpy.datetime64('2019-09-20T00:00:00.000000000'),
 numpy.datetime64('2019-12-20T00:00:00.000000000'),
 numpy.datetime64('2018-03-16T00:00:00.000000000'),
 numpy.datetime64('2018-06-15T00:00:00.000000000'),
 numpy.datetime64('2018-09-21T00:00:00.000000000'),
 numpy.datetime64('2018-12-21T00:00:00.000000000'),
 numpy.datetime64('2017-03-17T00:00:00.000000000'),
 numpy.datetime64('2017-06-16T00:00:00.000000000'),
 numpy.datetime64('2017-09-15T00:00:00.000000000'),
 numpy.datetime64('2017-12-15T00:00:00.000000000'),
 numpy.datetime64('2016-03-18T00:00:00.000000000'),
 numpy.datetime64('2016-06-17T00:00:00.000000000'),
 numpy.datetime64('2016-09-16T00:00:00.000000000'),
 numpy.datet

In [219]:
columns_to_create = ['original_date', 'total', 'mean', 'std', 'up', 'count', 'max', 'min', 'sharpe', 'uppct']

# output = pd.DataFrame(columns=columns_to_create)
# output.set_index(['date', 'add/delete'])

# output_add = pd.DataFrame(columns=columns_to_create)
# output_delete = pd.DataFrame(columns=columns_to_create)
regular_add = pd.DataFrame(columns=columns_to_create)
regular_delete = pd.DataFrame(columns=columns_to_create)
ad_hoc_add = pd.DataFrame(columns=columns_to_create)
ad_hoc_delete = pd.DataFrame(columns=columns_to_create)

# output_add['date'] = pd.to_datetime(output_add['date'])
# output_add.set_index('date')
# output_delete['date'] = pd.to_datetime(output_delete['date'])
# output_delete.set_index('date')

In [ ]:
MAX_DATE_EXTENSION = 10
days_bef_aft = 10 # the range of days before and after effective date to pull from Yahoo finance
last_date = sp500_history.iloc[0, sp500_history.columns.get_loc('date')]
last_type = sp500_history.iloc[0, sp500_history.columns.get_loc('variable')]
last_changes = []
is_begin = True
is_regular = False

for idx, row in sp500_history.iterrows():
    
    # End date is exclusive, so need to increase by 1
    prices = pdr.get_data_yahoo(row['value'], start=row['date'] - timedelta(days=days_bef_aft), end=row['date'] + timedelta(days=days_bef_aft + 1))
    if len(prices) == 0:
        continue
        
    # Find the adjusted effective date 
    # which is one trading day before the effective (market opening) date
    adj_date = row['date'] - timedelta(days=1)
    date_extension = 0
    while not adj_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        adj_date -= timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue
    adj_date_close = prices.loc[adj_date]['Close']
    
    if adj_date in rebal_dates:
        is_regular = True
    else:
        is_regular = False
        
    # Find the entry date, which is one trading day before the adjusted effective date
    prev_date = adj_date - timedelta(days=1)
    date_extension = 0
    while not prev_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        prev_date -= timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue
    prev_close = prices.loc[prev_date]['Close']

    # Find the next opening price
#     next_date = row['date']
#     date_extension = 0
#     while not next_date in prices.index and date_extension < MAX_DATE_EXTENSION:
#         next_date += timedelta(days=1)
#         date_extension += 1
#     if date_extension >= MAX_DATE_EXTENSION:
#         continue    
#     next_open = prices.loc[next_date]['Open']

    price_change = adj_date_close - prev_close

    # Calculate mean and all
    if not is_begin and (last_date != adj_date or last_type != row['variable']):
        print(last_changes)
        count = len(last_changes)
        mean = last_row['total'] / count
        if count > 1:
            std = statistics.stdev(last_changes)
        else:
            std = 0.000001
        up = len(list(filter(lambda x: (x >= 0), last_changes))) 
        max_val = max(last_changes)
        min_val = min(last_changes)
        sharpe = mean / std * np.sqrt(252)
        uppct = up / count

        data = [mean, std, up, count, max_val, min_val, sharpe, uppct]

        if last_type == 'added_ticker':
            if is_regular:
                regular_add.loc[last_date, 'mean':] = data
            else:
                ad_hoc_add.loc[last_date, 'mean':] = data

        else:
            if is_regular:
                regular_delete.loc[last_date, 'mean':] = data
            else:
                ad_hoc_delete.loc[last_date, 'mean':] = data

        total = 0
        last_changes = []
    
    # Still in the same date
    else:
        if is_begin:
            total = 0
            is_begin = False
        else:
            total = last_row['total']
    
    total += price_change
    
    data = {'total': total, 'original_date': row['date']}
            
    if row['variable'] == 'added_ticker':
        if is_regular:
            if adj_date not in regular_add.index:
                regular_add = regular_add.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                regular_add.loc[adj_date, 'total'] = total
            
        else:
            if adj_date not in ad_hoc_add.index:
                ad_hoc_add = ad_hoc_add.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                ad_hoc_add.loc[adj_date, 'total'] = total
                
    else:
        if is_regular:
            if adj_date not in regular_delete.index:
                regular_delete = regular_delete.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                regular_delete.loc[adj_date, 'total'] = total
            
        else:
            if adj_date not in ad_hoc_delete.index:
                ad_hoc_delete = ad_hoc_delete.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                ad_hoc_delete.loc[adj_date, 'total'] = total
                
    last_date = adj_date
    last_type = row['variable']
    last_changes.append(price_change)
    
    if last_type == 'added_ticker':
        if is_regular:
            last_row = regular_add.loc[last_date, :]
        else:
            last_row = ad_hoc_add.loc[last_date, :]
    
    else:
        if is_regular:
            last_row = regular_delete.loc[last_date, :]
        else:
            last_row = ad_hoc_delete.loc[last_date, :]
        
regular_add.index.names = ['date']
ad_hoc_add.index.names = ['date']
regular_delete.index.names = ['date']
ad_hoc_delete.index.names = ['date']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[-2.270000457763672]
[*********************100%***********************]  1 of 1 completed
[-3.6821136474609375]
[*********************100%***********************]  1 of 1 completed
[-1.19000244140625]
[*********************100%***********************]  1 of 1 completed
[0.0]
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRCM: No data found for this date range, symbol may be delisted


In [212]:
adj_date

Timestamp('2017-03-17 00:00:00')

In [186]:
output_add.to_csv('sp500_add.csv')
output_delete.to_csv('sp500_delete.csv')